In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import io as io
import math
import copy
import scipy.signal as signal

rcdef = plt.rcParams.copy()
newparams_for_screen = {'axes.labelsize': 14, 'axes.linewidth': 0.75, 
                        'savefig.dpi': 100, 'lines.linewidth': 0.6, 
                        'figure.figsize': (8, 6), 'ytick.labelsize': 12, 
                        'xtick.labelsize': 12, 'axes.titlesize': 14,
                        'legend.fontsize': 10, 'legend.frameon': True,
                        'legend.handlelength': 1.5}
plt.rcParams.update(newparams_for_screen)

---

### Exercise 4.1 
Extend your 1-d finite-volume solver for advection (from Exercise 3.7) to solve Burgers' equation. Your will need to change the Riemann solver and use the local velocity in the construction of the interface states. Run the examples shown in Figures 4.3 and 4.4.